In [1]:
def get_connected(market, trade_type):
    ''' get connected to ibkr
    Args: 
       (market) as string <'nse'> | <'snp'>
       (trade_type) as string <'live'> | <'paper'>
    Returns:
        (ib) object if successful
    '''
    
    ip = (market.upper(), trade_type.upper())
    
    #host dictionary
    hostdict = {('NSE', 'LIVE'): 3000,
                ('NSE', 'PAPER'): 3001,
                ('SNP', 'LIVE'): 1300,
                ('SNP', 'PAPER'): 1301,}
    
    host = hostdict[ip]
    
    cid = 1 # initialize clientId
    max_cid = 5 # maximum clientId allowed. max possible is 32

    for i in range(cid, max_cid):
        try:
            ib = IB().connect('127.0.0.1', host, clientId=i)
            
        except Exception as e:
            print(e) # print the error
            continue # go to next
            
        break # successful try
        
    return ib

In [2]:
from ib_insync import *
util.startLoop()
ib = get_connected('snp', 'live')

In [3]:
from itertools import repeat
def get_margins(ib, contracts, *lotsize):
    '''Margin dictionary. 1 min for 100 contracts.
    Args:
        (ib) as object
        (contracts) as list of underlying contract
        (*lotsize) as <int>|<list>
    Returns:
        {contract (obj): underlying_margin(float)} as dictionary'''
    
    if type(contracts) is pd.Series:
        contracts = list(contracts)
    else:
        contracts = contracts

    if type(lotsize[0]) is pd.Series:
        positions = list(lotsize[0])
    else:
        positions = repeat(lotsize[0], len(contracts)) # convert *arg tuple to int
    
    orders = [Order(action='SELL', orderType='MKT', totalQuantity=abs(p), whatIf=True)
              if p < 0 else
              Order(action='BUY', orderType='MKT', totalQuantity=abs(p), whatIf=True)
              for p in positions]

    co = [c for c in zip(contracts, orders)]

    dict_margins = {c: float(ib.whatIfOrder(c, o).initMarginChange) for c, o in co}
    
    return dict_margins

In [4]:
import pandas as pd
def get_underlyings(ib):
    '''Returns: list of underlying contracts
    Usage: 
       with get_connected('snp', 'live') as ib: und_contracts = get_underlyings(ib)'''

    # exclusion list
    excl = ['VXX','P', 'TSRO']

    # Download cboe weeklies to a dataframe
    dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

#     snp500 = list(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][1:].loc[:, 1])

    snp100 = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2][1:].loc[:, 0]
    snp100 = [s.replace('.', ' ') if '.' in s else s  for s in snp100] # without dot in symbol
    # read from row no 11, dropna and reset index
    df_cboe = pd.read_excel(dls, header=12, 
                            usecols=[0,2,3]).loc[11:, :]\
                            .dropna(axis=0)\
                            .reset_index(drop=True)

    # remove column names white-spaces and remap to IBKR
    df_cboe.columns = df_cboe.columns.str.replace(' ', '')

    # remove '/' for IBKR
    df_cboe.Ticker = df_cboe.Ticker.str.replace('/', ' ', regex=False)

    # make symbols
    symbols = {s for s in df_cboe.Ticker if s not in excl if s in snp100}
    stocks = [Stock(symbol=s, exchange='SMART', currency='USD') for s in symbols]

    blk = 50 # no of stocks in a block
    stkblks = [stocks[i: i+blk] for i in range(0, len(stocks), blk)] # blocks of stocks

    # qualify the contracts
    contracts = [ib.qualifyContracts(*s) for s in stkblks]

    # return flattened contract list
    return [contract for subl in contracts for contract in subl] 

In [5]:
from math import sqrt

from os import listdir, unlink, path
from datetime import datetime, timedelta

tradingdays = 252

hrs = 24 # setting time to check freshness of ohlc

def get_ohlc(ib, contracts, fspath):
    '''Gets the ohlc
    Args: 
       (ib) as ib object
       (contracts) as list of qualified contracts
       (fspath) as string containing zdata path
    Returns:
       None. But pickles ohlc'''
    
    fs = [f for f in listdir(fspath) if (f[-8:] == 'ohlc.pkl')] # list of opt pickle files

    floortime = (datetime.now() - timedelta(hours = hrs)).timestamp()

    if fs:
        lasttime = min([v for k, v in {f: path.getmtime(fspath + f) for f in fs}.items()])

    else: # if the folder is empty
        lasttime = (datetime.now() - timedelta(hours = hrs+1)).timestamp() # make last time older than 24 hours

    # generate OHLC pickles if data is older than 24 hours or no data
    if lasttime < floortime:
        
        for contract in contracts:
    
            # Extract the history
            hist = ib.reqHistoricalData(contract=contract, endDateTime='', 
                                            durationStr='365 D', barSizeSetting='1 day',  
                                                        whatToShow='Trades', useRTH=True)
            df = util.df(hist)
            df.insert(0, column='symbol', value=contract.symbol)

            df_ohlc = df.set_index('date').sort_index(ascending = False)

            # get cumulative standard deviation
            df_stdev = pd.DataFrame(df_ohlc['close'].expanding(1).std(ddof=0))
            df_stdev.columns = ['stdev']

            # get cumulative volatility
            df_vol = pd.DataFrame(df_ohlc['close'].pct_change().expanding(1).std(ddof=0)*sqrt(tradingdays))
            df_vol.columns = ['volatility']

            df_ohlc1 = df_ohlc.join(df_vol)

            df_ohlc2 = df_ohlc1.join(df_stdev)

            #pickle the ohlc
            df_ohlc2.to_pickle(fspath+contract.symbol+'_ohlc.pkl')
    
    return None

In [8]:
from math import sqrt

from os import listdir, unlink, path
from datetime import datetime, timedelta

tradingdays = 252

hrs = 24 # setting time to check freshness of ohlc

async def ohlcAsync(ib, contracts, fspath):
    '''Gets the ohlc
    Args: 
       (ib) as ib object
       (contracts) as list of qualified contracts
       (fspath) as string containing zdata path
    Returns:
       None. But pickles ohlc'''
    
    fs = [f for f in listdir(fspath) if (f[-8:] == 'ohlc.pkl')] # list of opt pickle files

    floortime = (datetime.now() - timedelta(hours = hrs)).timestamp()

    if fs:
        lasttime = min([v for k, v in {f: path.getmtime(fspath + f) for f in fs}.items()])

    else: # if the folder is empty
        lasttime = (datetime.now() - timedelta(hours = hrs+1)).timestamp() # make last time older than 24 hours

    # generate OHLC pickles if data is older than 24 hours or no data
    if lasttime < floortime:
        
        for contract in contracts:
    
            # Extract the history
            hist = ib.reqHistoricalData(contract=contract, endDateTime='', 
                                            durationStr='365 D', barSizeSetting='1 day',  
                                                        whatToShow='Trades', useRTH=True)
            df = util.df(hist)
            df.insert(0, column='symbol', value=contract.symbol)

            df_ohlc = df.set_index('date').sort_index(ascending = False)

            # get cumulative standard deviation
            df_stdev = pd.DataFrame(df_ohlc['close'].expanding(1).std(ddof=0))
            df_stdev.columns = ['stdev']

            # get cumulative volatility
            df_vol = pd.DataFrame(df_ohlc['close'].pct_change().expanding(1).std(ddof=0)*sqrt(tradingdays))
            df_vol.columns = ['volatility']

            df_ohlc1 = df_ohlc.join(df_vol)

            df_ohlc2 = df_ohlc1.join(df_stdev)

            #pickle the ohlc
            df_ohlc2.to_pickle(fspath+contract.symbol+'_ohlc.pkl')
    
    return None

In [6]:
%%time
#____________________________________________________________________________________

# This section needs to be a script.
# It needs to be run once first to get df_p and remaining quantities

import numpy as np

if not ib.isConnected():
    ib = get_connected('snp', 'live')

m_maxp = 0.02    # % of max margin allowed on net liquidity per scrip to limit positon risk
max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

fspath = '../data/snp/' # path for pickles
       
#...get current positions
#________________________

#... read the account info
ac = ib.accountValues()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
# av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
strike = [d.strike for d in df_p.contract]

df_p['symbol'] = [s.symbol for s in df_p.contract.values]
df_p['expiry'] = expiry
df_p['strike'] = strike

df_p = df_p.assign(type=[type(x).__name__ for x in df_p.contract])

# get the contracts again (as some of them miss markets)
port_c = [j for c in [ib.qualifyContracts(Contract(conId=c.conId)) for c in df_p.contract] for j in c]

df_p = df_p.assign(contract=port_c)

# get the margins
dict_port_opt_margins = get_margins(ib, df_p.contract, df_p.position)
df_p = df_p.assign(totmargin=df_p.contract.map(dict_port_opt_margins))

df_p = df_p.assign(lotsize=[int(k.multiplier) for k in port_c])

df_p_margin = pd.DataFrame(df_p.groupby('symbol').sum().totmargin).reset_index()

#... get the underlying margins and the blacklist
#____________________________________________________

undContracts = get_underlyings(ib)
lotsize=100
undMargins = get_margins(ib, undContracts, 100)
undMargins = {k.symbol: v for k, v in undMargins.items()} # margin with symbols instead of contracts
undMargins = {k: abs(v) for k, v in undMargins.items()} # positive margins

df_p_margin = df_p_margin.assign(margin = df_p_margin.symbol.map(undMargins))
df_p_margin = df_p_margin.assign(maxmargin=max_p)
df_p_margin = df_p_margin.assign(netqty=((df_p_margin.maxmargin-df_p_margin.totmargin)/df_p_margin.margin).apply(np.floor))
df_p_margin = df_p_margin.assign(remqty=np.where(df_p_margin.netqty > 0, df_p_margin.netqty, 0).astype(int))

blacklist = list(df_p_margin[df_p_margin.remqty <= 0 ].symbol)

#...get the max remaining quantities for each symbol
masterContracts = [c for c in undContracts if c.symbol not in blacklist]

Wall time: 51.5 s


In [9]:
#****!!! DATA LIMITER*****
masterContracts = masterContracts[:5]

masterMargin = {m.symbol: undMargins[m.symbol] for m in masterContracts}

maxqty = {k: int(max_p/m) for k, m in masterMargin.items()}
remq = list(df_p_margin[['symbol', 'remqty']].set_index('symbol').to_dict().values())[0]
maxremqty = {k: remq.get(k, v) for k, v in maxqty.items()}

In [10]:
%%time
#____________________________________________________________________________________

#...get the ohlcs


task1 = await ohlcAsync(ib=ib, contracts=masterContracts, fspath=fspath)

SyntaxError: 'await' outside function (<timed exec>, line 4)

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
An existing connection was forcibly closed by the remote host


In [ ]:
#...get the underlying prices
tickers = await ib.reqTickersAsync(*masterContracts)

In [ ]:
#...get the chains
for c in masterContracts:
    chains =  await ib.reqSecDefOptParamsAsync(underlyingSymbol=c.symbol, futFopExchange='', 
                                  underlyingConId=c.conId, underlyingSecType=c.secType)
    util.df(chains).to_pickle(fspath+c.symbol+'_chains.pkl')

In [ ]:
a

In [ ]:
undPrice = {t.contract.symbol: t.marketPrice() for t in tickers}

In [ ]:
# %%time
#____________________________________________________________________________________


In [ ]:
import pandas as pd
def get_rfrate():
    '''Returns: risk Free rate for the market as float'''
    rate_url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield'
    df_rate = pd.read_html(rate_url)[1]
    df_rate.columns  = df_rate.iloc[0] # Set the first row as header
    df_rate = df_rate.drop(0,0) # Drop the first row
    rate = float(df_rate[-1:]['1 yr'].values[0])/100 # Get the last row's 1 yr value as float
    return rate


In [ ]:
def get_dividend_tick(ib, contract):
    ''' Gets the dividend ticker
    Args:
       (ib) as the connected object
       (contract) as the qualified contract object with conId
    '''
    ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False) # request ticker stream
    ticker_data = ib.ticker(contract)
    
    while ticker_data.dividends is None:
        ib.sleep(0.1)
        
    ib.cancelMktData(contract)
    
    return ticker_data

In [ ]:
# gets days to expiry from now onwards
import datetime

def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format 'yyyymmdd'
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days

In [ ]:
symbol = snp[5]

from itertools import product

ib=get_connected('snp', 'live')

und_contract = ib.qualifyContracts(Stock(symbol=symbol, exchange='SMART', currency='USD'))[0]

es = list([product(c.expirations, c.strikes) 
        for c in ib.reqSecDefOptParams(underlyingSymbol=und_contract.symbol, 
                               futFopExchange='', underlyingSecType=und_contract.secType, 
                               underlyingConId=und_contract.conId)
        if c.exchange == 'SMART'
        if c.expirations
        if c.strikes][0])

In [ ]:
ib.disconnect()

In [ ]:
df_ohlc = get_ohlc(ib, und_contract, fspath)

In [ ]:
df_ohlc[:45].stdev.max()*3

In [ ]:
    und_contracts = ib.qualifyContracts(*(Stock(symbol=s, exchange='SMART', currency='USD') for s in snp))
    tickers = ib.reqTickers(*und_contracts)
    und_prices = {t.contract.symbol: t.marketPrice() for t in tickers}
    div_ticks = [get_dividend_tick(ib, contract) for contract in und_contracts]

In [ ]:
with get_connected('snp', 'live') as ib:
    df_ohlc = get_ohlc(ib, und_contracts[20], fspath)

In [ ]:
from itertools import product

with get_connected('snp', 'live') as ib:
    chains = [ib.reqSecDefOptParams(underlyingSymbol=und_contract.symbol, futFopExchange='', 
                                  underlyingConId=und_contract.conId, underlyingSecType=und_contract.secType) for und_contract in und_contracts]

In [ ]:
chains